 En esta sección, se describe cómo se limpiaron y prepararon los datos para su análisis, incluyendo la eliminación de valores atípicos y la imputación de datos faltantes.

In [2]:
import json
import pandas as pd
from nltk.corpus import stopwords
import contractions
import re
import nltk

In [20]:
file_path = "../data/tickets_classification_eng.json"

# Leer el archivo JSON 
with open(file_path, "r", encoding="utf-8") as file:  
    datos = json.load(file)

df = pd.json_normalize(datos)

In [21]:
columns_to_select = [
    "_source.complaint_what_happened",
    "_source.product",
    "_source.sub_product"
]

df_selected = df[columns_to_select]

In [22]:
# Renombrar columnas para mayor claridad
df_selected.rename(columns={
    "_source.complaint_what_happened": "complaint_what_happened",
    "_source.product": "category",
    "_source.sub_product": "sub_product"
}, inplace=True)

C:\Users\Francisco\AppData\Local\Temp\ipykernel_3080\3935133602.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected.rename(columns={


In [9]:
df_selected.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78313 entries, 0 to 78312
Data columns (total 3 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   complaint_what_happened  78313 non-null  object
 1   category                 78313 non-null  object
 2   sub_product              67742 non-null  object
dtypes: object(3)
memory usage: 1.8+ MB


### Creación de Nueva Columna:

 Añade una nueva columna llamada ticket_classification que sea el resultado de concatenar los valores de las columnas category y sub_product, separados por un signo más. Por ejemplo, si category contiene "Banco" y sub_product contiene "Cuenta Corriente", entonces ticket_classification debería ser "Banco + Cuenta Corriente".

In [23]:
df_selected["ticket_classification"] = df_selected["category"] + " + " + df_selected["sub_product"]

C:\Users\Francisco\AppData\Local\Temp\ipykernel_3080\766719112.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected["ticket_classification"] = df_selected["category"] + " + " + df_selected["sub_product"]


In [24]:
df_selected.head()

,complaint_what_happened,category,sub_product,ticket_classification
0,,Debt collection,Credit card debt,Debt collection + Credit card debt
1,Good morning my name is XXXX XXXX and I apprec...,Debt collection,Credit card debt,Debt collection + Credit card debt
2,I upgraded my XXXX XXXX card in XX/XX/2018 and...,Credit card or prepaid card,General-purpose credit card or charge card,Credit card or prepaid card + General-purpose ...
3,,Mortgage,Conventional home mortgage,Mortgage + Conventional home mortgage
4,,Credit card or prepaid card,General-purpose credit card or charge card,Credit card or prepaid card + General-purpose ...


### Eliminar Columnas Redundantes:

Después de crear la columna ticket_classification, elimina las columnas sub_product y category, ya que su información ahora está encapsulada en la nueva columna.

In [25]:
df_selected.drop(columns=["sub_product", "category"], inplace=True)

C:\Users\Francisco\AppData\Local\Temp\ipykernel_3080\78379480.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected.drop(columns=["sub_product", "category"], inplace=True)


### Limpieza de Datos en Columnas Específicas: 

Aquí aseguramos de que la columna complaint_what_happened no contenga campos vacíos. Reemplaza esos campos vacíos con un valor que indique que los datos están ausentes (como NaN).

In [26]:
df_selected["complaint_what_happened"].replace("", pd.NA, inplace=True)

C:\Users\Francisco\AppData\Local\Temp\ipykernel_3080\1068415345.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_selected["complaint_what_happened"].replace("", pd.NA, inplace=True)
C:\Users\Francisco\AppData\Local\Temp\ipykernel_3080\1068415345.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected["complaint_what_happened"].re

### Eliminación de Filas con Datos Faltantes: 

Elimina todas las filas que tengan datos faltantes en las columnas críticas, es decir, complaint_what_happened y ticket_classification.

In [27]:
df_selected.dropna(subset=["complaint_what_happened", "ticket_classification"], inplace=True)

C:\Users\Francisco\AppData\Local\Temp\ipykernel_3080\561902098.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected.dropna(subset=["complaint_what_happened", "ticket_classification"], inplace=True)


### Limpiar las "X" con nltk y eliminar stopwords para después guardar el modelo 

In [43]:
# Descargar las stopwords de nltk si no lo has hecho
nltk.download('stopwords')
nltk.download('punkt')

# Definir las stopwords en inglés
stopwords_ingles = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Francisco\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Francisco\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [44]:
def limpiar_texto(texto):
    """
    Función para limpiar y preprocesar un texto.
    """
    # Paso 1: Expandir contracciones
    texto_expandido = contractions.fix(texto)

    # Paso 2: Remover caracteres 'X' repetidos que simbolizan datos confidenciales
    texto_sin_x = re.sub(r'X+', '', texto_expandido)

    # Paso 3: Convertir a minúsculas
    texto_minusculas = texto_sin_x.lower()

    # Paso 4: Tokenizar
    tokens = nltk.word_tokenize(texto_minusculas)

    # Paso 5: Filtrar stopwords
    tokens_filtrados = filter(lambda palabra: palabra not in stopwords_ingles, tokens)

    # Paso 6: Unir palabras filtradas
    texto_procesado = ' '.join(tokens_filtrados)

    return texto_procesado


In [41]:
# Aplicar la función a la columna 'complaint_what_happened'
df_selected['complaint_what_happened'] = df_selected['complaint_what_happened'].apply(limpiar_texto)

C:\Users\Francisco\AppData\Local\Temp\ipykernel_3080\1786559717.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['complaint_what_happened'] = df_selected['complaint_what_happened'].apply(limpiar_texto)


In [45]:
df_selected

,complaint_what_happened,ticket_classification
1,good morning name appreciate could help put st...,Debt collection + Credit card debt
2,upgraded card //2018 told agent upgrade annive...,Credit card or prepaid card + General-purpose ...
10,"chase card reported //2019 . however , fraudul...","Credit reporting, credit repair services, or o..."
11,"//2018 , trying book ticket , came across offe...","Credit reporting, credit repair services, or o..."
14,grand son give check { $ 1600.00 } deposit cha...,Checking or savings account + Checking account
...,...,...
78301,husband passed away . chase bank put check hol...,Checking or savings account + Checking account
78303,"chase card customer well decade , offered mult...",Credit card or prepaid card + General-purpose ...
78309,"wednesday , // called chas , visa credit card ...",Credit card or prepaid card + General-purpose ...
78310,familiar pay understand great risk provides co...,Checking or savings account + Checking account


In [46]:
##### Ahora si ya más limpio falta reiniciar el índex y guardarlo 
df_selected.reset_index(drop=True, inplace=True)

In [47]:
output_path = "../data/processed/tickets_cleaned.csv"

In [48]:
df_selected.to_csv(output_path, index=False)
print(f"DataFrame guardado exitosamente en: {output_path}")

DataFrame guardado exitosamente en: ../data/processed/tickets_cleaned.csv


## Posibles modelos a utilizar

####  Regresión Logística 

Por qué usarla
* Modelo lineal: La regresión logística es un modelo estadístico sencillo que asume una relación lineal entre las variables independientes (predictoras) y la probabilidad de que una observación pertenezca a una clase específica.


* Interpretabilidad: Es fácil de interpretar, ya que los coeficientes indican la contribución de cada característica al resultado.


* Rápido y eficiente: Es computacionalmente eficiente, lo que lo hace ideal como punto de partida para problemas de clasificación.

#### Random Forest 

Por qué usarla

* Modelo no lineal: Random Forest es un modelo basado en árboles de decisión, que puede capturar relaciones complejas y no lineales entre las características.


* Versatilidad: Funciona bien tanto con datos categóricos como continuos, lo que lo hace adecuado para conjuntos de datos mixtos.


* Robustez: Es resistente al sobreajuste debido al ensamble de múltiples árboles (votación de mayorías en clasificación).

#### Support Vector Classifier 

Por qué usarlo

* Separación óptima: SVC busca el hiperplano que maximiza la separación entre las clases, lo que lo hace efectivo para problemas con datos bien separables.


* Adaptabilidad: Con el uso de diferentes funciones kernel (linear, rbf), SVC puede adaptarse a problemas lineales y no lineales.


* Robustez en problemas con ruido: Es eficaz en conjuntos de datos pequeños y con clases desbalanceadas, ya que se enfoca en los puntos más críticos (los cercanos al margen).

Posible implementación y análisis: 

Iniciar con Regresión Logística como un modelo base para evaluar la dificultad del problema.
Usar Random Forest si hay sospechas de relaciones no lineales o características más complejas.
Explorar el SVC si el tamaño del conjunto de datos lo permite y se requieren soluciones más sofisticadas.